# Couler Examples

## Prepare your system

In [ ]:
NAMESPACE = "<your-namespace>"

In [ ]:
!pip config --user set global.index-url https://jfrog.aaw.cloud.statcan.ca/artifactory/api/pypi/pypi-remote/simple
!python3 -m pip install git+https://github.com/couler-proj/couler --ignore-installed

## Whalesay!

## Import the appropriate libraries

In [ ]:
import json
import random

import couler.argo as couler
from couler.argo_submitter import ArgoSubmitter

In [ ]:
## A simple example with cowsay.

couler.run_container(
    image="docker/whalesay",
    command=["cowsay"],
    args=["hello world"]
)


submitter = ArgoSubmitter(namespace=NAMESPACE)
result = couler.run(submitter=submitter)

print(json.dumps(result, indent=2))

## A slightly more complex example.

In [ ]:
def random_code():
    import random
    res = "heads" if random.randint(0, 1) == 0 else "tails"
    print(res)


def flip_coin():
    return couler.run_script(image="python:alpine3.6", source=random_code)


def heads():
    return couler.run_container(
        image="alpine:3.6", command=["sh", "-c", 'echo "it was heads"']
    )


def tails():
    return couler.run_container(
        image="alpine:3.6", command=["sh", "-c", 'echo "it was tails"']
    )


result = flip_coin()

couler.when(couler.equal(result, "heads"), lambda: heads())
couler.when(couler.equal(result, "tails"), lambda: tails())

submitter = ArgoSubmitter(namespace="<your-kubeflow-namespace>")
result = couler.run(submitter=submitter)

print(json.dumps(result, indent=2))

## The above Couler code generates the following Kubernetes YAML

## An example with a directed acyclic graph (DAG).

In [ ]:
def job(name):
    couler.run_container(
        image="docker/whalesay:latest",
        command=["cowsay"],
        args=[name],
        step_name=name,
    )


#     A
#    / \
#   B   C
#  /
# D
def linear():
    couler.set_dependencies(lambda: job(name="A"), dependencies=None)
    couler.set_dependencies(lambda: job(name="B"), dependencies=["A"])
    couler.set_dependencies(lambda: job(name="C"), dependencies=["A"])
    couler.set_dependencies(lambda: job(name="D"), dependencies=["B"])


#   A
#  / \
# B   C
#  \ /
#   D
def diamond():
    couler.dag(
        [
            [lambda: job(name="A")],
            [lambda: job(name="A"), lambda: job(name="B")],  # A -> B
            [lambda: job(name="A"), lambda: job(name="C")],  # A -> C
            [lambda: job(name="B"), lambda: job(name="D")],  # B -> D
            [lambda: job(name="C"), lambda: job(name="D")],  # C -> D
        ]
    )


linear()
submitter = ArgoSubmitter(namespace="<your-kubeflow-namespace>")
result = couler.run(submitter=submitter)

print(json.dumps(result, indent=2))